<a href="https://colab.research.google.com/github/ameyas1/Anna_Karenina_Text_Generation/blob/master/anna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [0]:
# open text file and read in data as `text`
with open('book_anna.txt', 'r') as f:
    text = f.read()
text=text.lower()

In [3]:
text[:100]

'chapter 1\n\n\nhappy families are all alike; every unhappy family is unhappy in its own\nway.\n\neverythin'

In [0]:
characters=sorted(list(set(text)))
n_to_char={n:char for n, char in enumerate(characters)}
char_to_n={char:n for n, char in enumerate(characters)}


In [0]:
encoded = np.array([char_to_n[ch] for ch in text])

In [6]:
encoded[:100]

array([27, 32, 25, 40, 44, 29, 42,  1, 11,  0,  0,  0, 32, 25, 40, 40, 49,
        1, 30, 25, 37, 33, 36, 33, 29, 43,  1, 25, 42, 29,  1, 25, 36, 36,
        1, 25, 36, 33, 35, 29, 21,  1, 29, 46, 29, 42, 49,  1, 45, 38, 32,
       25, 40, 40, 49,  1, 30, 25, 37, 33, 36, 49,  1, 33, 43,  1, 45, 38,
       32, 25, 40, 40, 49,  1, 33, 38,  1, 33, 44, 43,  1, 39, 47, 38,  0,
       47, 25, 49,  9,  0,  0, 29, 46, 29, 42, 49, 44, 32, 33, 38])

In [0]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)

    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [0]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [9]:
encoded.shape

(1966144,)

In [10]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [0]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=3,
                               drop_prob=0.3, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [0]:
def train(net, data, epochs=20, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [13]:
n_hidden=512
n_layers=3

net = CharRNN(characters, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(51, 512, num_layers=3, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3)
  (fc): Linear(in_features=512, out_features=51, bias=True)
)


In [14]:
batch_size = 16
seq_length = 991
n_epochs = 40 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/40... Step: 10... Loss: 3.0987... Val Loss: 3.0578
Epoch: 1/40... Step: 20... Loss: 3.0379... Val Loss: 3.0234
Epoch: 1/40... Step: 30... Loss: 3.0460... Val Loss: 3.0209
Epoch: 1/40... Step: 40... Loss: 3.0267... Val Loss: 3.0180
Epoch: 1/40... Step: 50... Loss: 3.0462... Val Loss: 3.0167
Epoch: 1/40... Step: 60... Loss: 3.0115... Val Loss: 3.0156
Epoch: 1/40... Step: 70... Loss: 3.0077... Val Loss: 3.0160
Epoch: 1/40... Step: 80... Loss: 3.0105... Val Loss: 3.0141
Epoch: 1/40... Step: 90... Loss: 3.0403... Val Loss: 3.0104
Epoch: 1/40... Step: 100... Loss: 3.0358... Val Loss: 3.0012
Epoch: 1/40... Step: 110... Loss: 3.0161... Val Loss: 3.0377
Epoch: 2/40... Step: 120... Loss: 3.0027... Val Loss: 2.9755
Epoch: 2/40... Step: 130... Loss: 2.9165... Val Loss: 2.8783
Epoch: 2/40... Step: 140... Loss: 2.8476... Val Loss: 2.7931
Epoch: 2/40... Step: 150... Loss: 2.7847... Val Loss: 2.7580
Epoch: 2/40... Step: 160... Loss: 2.7704... Val Loss: 2.7278
Epoch: 2/40... Step: 170... Loss:

In [0]:
# change the name, for saving multiple files
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [0]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [0]:
def sample(net, size, prime='the', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [18]:
print(sample(net, 1000, prime='and levin said', top_k=2))

and levin said: "i've been at that moment," said stepan
arkadyevitch, when she held her hand and smiled to her.

"well, i'm not going to be something there are a special
significance. the princess was a special sensation of prints."

"i have told you," he thought, and still made haste for him.

"well, what are you told to? what is it? i don't know what i would be a good man, if you want to see you," said stepan arkadyevitch, smiling.

"i haven't
the consequence of my
position," said
levin, and she saw the
conversation. "i have never seen him and
say a great deal to see you," said levin, as he walked on, and smiled to his wife.

"i don't understand it," she said with a slight smile, and
was silent. she stopped short to her.

"well, i won't see, to be some money to be sure, i don't know what you want to be done. i see it all the same to be sure. i don't understand," he said,
and was already because he
was not a silence, and then he was
standing at his brother, and he walked away from him